# 2 - Heuristique TSE
On s'intéresse à savoir :
- si une interaction du type $X_i \circ X_j$ pour prédire une cible peut être créée (où $\circ$ désigne n'importe quel opérateur arithmétique parmi {+, -, *, /})
- si cette interaction est logique et *suffisamment* significative

**Exemple : **
1. on teste l'interaction $x_2 \times x_{13}$ sur une cible. 
1. Règle du pouce : si la corrélation entre $x_2 \times x_{13}$ est plus grande que la plus grande corrélation existante$^*$ du dataset, on créé la feature. 
1. On vérifie que l'interaction créée est significative : 
    - avec une logique métier (prior)
    - sur un échantillon de validation, pour une corrélation de <a href='https://en.wikipedia.org/wiki/Pearson_correlation_coefficient'>Pearson</a>, et pour une corrélation de <a href='https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient'>Spearman</a>
1. Les interactions significatives sur ces 3 critères peuvent être ajoutées à un modèle.

$^*$Cette corrélation peut être mesurée avec le coefficient de Pearson (si les données sont normales, rapide), ou avec le coefficient de Spearman (non paramétrique, lent). 

**Remarques :**
- Les interactions ne peuvent être créées qu'avec des types numériques (entier, flottants, booléens). 
- Les *nan* sont tolérés, mais ne serviront pas au calcul des corrélations.
- Plus un dataset est "facile" à prédire (i.e. les $X_i$ sont fortement corrélés à la cible), plus le paramètre `marge` devra être élevé.

*Author: B011LDG*

### Import et préparation des données

In [1]:
import pandas as pd ; pd.set_option("max_columns", 60)
import numpy as np
from Heuristique_TSE import *

In [2]:
train = pd.read_excel('data_Femmes.xls')
train.origine = np.where(train.origine=='européennes', 1, 0)
train.rename(columns={'origine':'from_europ'}, inplace=True)

train.tabac = train.tabac.map({'ne tolère pas la fumée':0, 'tolère la fumée':0, 'en soirée':1,
                               'souvent':2, "j'essaie d'arrêter":1, 'de temps en temps':1, np.nan:0})

train = train.select_dtypes(('float', 'int', 'bool')).astype('float')
print(train.shape)
train.head()

(532, 23)


,age,charmes,identifiant,label,mails,from_europ,paniers,poids,tabac,taille,lunettes,tatouage,piercing,accessibilité,long_description,nbMots_description,IMC,a_redoublé,nb_series_communes,distanceToMe,est_de_nuit,may_be_maman,pct_complétion
0,23.0,74.0,115353179.0,1.0,0.0,1.0,0.0,NaN,0.0,165.0,0.0,0.0,0.0,0.000000,367.0,64.0,NaN,0.0,0.0,30.932590,0.0,0.0,0.750000
1,25.0,337.0,113878261.0,1.0,15.0,0.0,4.0,55.0,0.0,165.0,0.0,0.0,0.0,0.044510,0.0,0.0,20.20202,0.0,0.0,18.980000,0.0,0.0,0.750000
2,26.0,475.0,114925419.0,1.0,28.0,1.0,0.0,50.0,0.0,160.0,1.0,0.0,0.0,0.058947,0.0,0.0,19.53125,0.0,1.0,21.195632,0.0,0.0,0.791667
3,20.0,144.0,114856526.0,1.0,10.0,0.0,0.0,NaN,0.0,160.0,0.0,0.0,0.0,0.069444,391.0,64.0,NaN,0.0,1.0,10.500000,0.0,0.0,0.833333
4,25.0,38.0,114532012.0,1.0,6.0,1.0,10.0,NaN,0.0,165.0,1.0,0.0,0.0,0.157895,348.0,60.0,NaN,0.0,2.0,24.106789,0.0,0.0,0.875000


In [3]:
# Normalisation : 
train = (train - train.mean()) / (train.std())

In [4]:
train.head()

,age,charmes,identifiant,label,mails,from_europ,paniers,poids,tabac,taille,lunettes,tatouage,piercing,accessibilité,long_description,nbMots_description,IMC,a_redoublé,nb_series_communes,distanceToMe,est_de_nuit,may_be_maman,pct_complétion
0,-0.588369,-0.367834,0.430220,1.308076,-0.409347,0.675777,-0.431616,NaN,-0.452887,0.208323,-0.599947,-0.50422,-0.417211,-0.781043,0.110280,0.085813,NaN,-0.226466,-0.620204,0.047254,-0.221849,-0.186959,-0.067167
1,0.436221,-0.241317,0.388134,1.308076,-0.329808,-1.476997,-0.287766,0.031658,-0.452887,0.208323,-0.599947,-0.50422,-0.417211,-0.572055,-0.790243,-0.788700,-0.057611,-0.226466,-0.620204,-0.244529,-0.221849,-0.186959,-0.067167
2,0.948517,-0.174932,0.418014,1.308076,-0.260874,0.675777,-0.431616,-0.366180,-0.452887,-0.533827,1.663682,-0.50422,-0.417211,-0.504270,-0.790243,-0.788700,-0.213197,-0.226466,0.789833,-0.190441,-0.221849,-0.186959,0.205603
3,-2.125256,-0.334160,0.416048,1.308076,-0.356321,-1.476997,-0.431616,NaN,-0.452887,-0.533827,-0.599947,-0.50422,-0.417211,-0.454984,0.169170,0.085813,NaN,-0.226466,0.789833,-0.451539,-0.221849,-0.186959,0.478374
4,0.436221,-0.385152,0.406788,1.308076,-0.377531,0.675777,-0.071992,NaN,-0.452887,0.208323,1.663682,-0.50422,-0.417211,-0.039688,0.063659,0.031156,NaN,-0.226466,2.199870,-0.119375,-0.221849,-0.186959,0.751144


In [5]:
# Obtention d'un échantillon d'entraînement et d'un échantillon de validation : 
from sklearn.model_selection import train_test_split
mini_train_X, valid_X, mini_train_y, valid_y = train_test_split(train.drop('label', 1), 
                                                                train.label, 
                                                                test_size=0.2)
mini_train_X.shape, valid_X.shape, mini_train_y.shape, valid_y.shape

((425, 22), (107, 22), (425,), (107,))

### Pearson

In [6]:
heuristique_p = Heuristique_TSE(mini_train_X, mini_train_y, 
                                corr_mode='pearson', interaction_mode="-", marge=0.05)
heuristique_p.make_interactions()

La corrélation (pearson) minimale avec la cible est -0.3611 ; la corrélation maximale avec la cible est 0.1989.
Itération 0/231 ;  23/231 ;  46/231 ;  69/231 ;  92/231 ;  115/231 ;  138/231 ;  161/231 ;  184/231 ;  207/231 ;  230/231 ;  

In [7]:
#heuristique_p.change_marge(0.1)
pearson_corr = heuristique_p.interactions_à_créer
pearson_corr, len(pearson_corr)

([('age', 'tabac', 0.2843),
  ('age', 'tatouage', 0.305),
  ('age', 'piercing', 0.2625),
  ('charmes', 'tabac', 0.2607),
  ('charmes', 'tatouage', 0.2778),
  ('mails', 'tabac', 0.2726),
  ('mails', 'tatouage', 0.2864),
  ('mails', 'piercing', 0.2527),
  ('from_europ', 'tabac', 0.3138),
  ('from_europ', 'tatouage', 0.3451),
  ('from_europ', 'piercing', 0.3021),
  ('paniers', 'tabac', 0.2527),
  ('paniers', 'tatouage', 0.269),
  ('lunettes', 'tatouage', 0.3944),
  ('lunettes', 'piercing', 0.3364)],
 15)

In [8]:
heuristique_p.valide_interaction(valid_X, valid_y, remove=True) # vérification que les interactions sont bonnes

///Corr. entre age et tabac = 0.1014, significative sur le train mais pas sur le valid !///
///Corr. entre age et piercing = 0.0113, significative sur le train mais pas sur le valid !///
///Corr. entre charmes et tatouage = 0.0461, significative sur le train mais pas sur le valid !///
///Corr. entre mails et tatouage = 0.0231, significative sur le train mais pas sur le valid !///
///Corr. entre from_europ et tabac = -0.0880, significative sur le train mais pas sur le valid !///
///Corr. entre from_europ et piercing = 0.0323, significative sur le train mais pas sur le valid !///
///Corr. entre paniers et tatouage = -0.0080, significative sur le train mais pas sur le valid !///
///Corr. entre lunettes et piercing = -0.0397, significative sur le train mais pas sur le valid !///


In [9]:
heuristique_p.interactions_à_créer

[('age', 'tatouage', 0.305),
 ('charmes', 'tabac', 0.2607),
 ('mails', 'tabac', 0.2726),
 ('mails', 'piercing', 0.2527),
 ('from_europ', 'tatouage', 0.3451),
 ('paniers', 'tabac', 0.2527),
 ('lunettes', 'tatouage', 0.3944)]

### Spearman
Si l'on n'est pas pressé, on peut vérifier que les interactions suggérées par Pearson sont également vérifiées par Spearman.

In [10]:
heuristique_sp = Heuristique_TSE(mini_train_X, mini_train_y, corr_mode='spearman', interaction_mode="-", marge=0.05)
heuristique_sp.make_interactions()

La corrélation (spearman) minimale avec la cible est -0.3668 ; la corrélation maximale avec la cible est 0.1764.
Itération 0/231 ;  23/231 ;  46/231 ;  69/231 ;  92/231 ;  115/231 ;  138/231 ;  161/231 ;  184/231 ;  207/231 ;  230/231 ;  

In [11]:
#heuristique_sp.change_marge(0.1)
spearman_corr = heuristique_sp.interactions_à_créer
spearman_corr, len(spearman_corr)

([('age', 'tabac', 0.2717),
  ('age', 'tatouage', 0.2974),
  ('age', 'piercing', 0.2449),
  ('charmes', 'tabac', 0.3001),
  ('charmes', 'tatouage', 0.3049),
  ('charmes', 'piercing', 0.2568),
  ('mails', 'tabac', 0.2536),
  ('mails', 'tatouage', 0.2706),
  ('from_europ', 'tabac', 0.2872),
  ('from_europ', 'tatouage', 0.3725),
  ('from_europ', 'piercing', 0.306),
  ('paniers', 'tabac', 0.2733),
  ('paniers', 'tatouage', 0.2768),
  ('lunettes', 'tatouage', 0.425),
  ('lunettes', 'piercing', 0.35)],
 15)

In [12]:
heuristique_sp.valide_interaction(X_valid=valid_X, y_valid=valid_y, remove=True)
heuristique_sp.interactions_à_créer

///Corr. entre age et tabac = 0.1014, significative sur le train mais pas sur le valid !///
///Corr. entre age et piercing = 0.0113, significative sur le train mais pas sur le valid !///
///Corr. entre charmes et tatouage = 0.0461, significative sur le train mais pas sur le valid !///
///Corr. entre mails et tabac = 0.1171, significative sur le train mais pas sur le valid !///
///Corr. entre from_europ et tabac = -0.0880, significative sur le train mais pas sur le valid !///
///Corr. entre from_europ et piercing = 0.0323, significative sur le train mais pas sur le valid !///
///Corr. entre paniers et tatouage = -0.0080, significative sur le train mais pas sur le valid !///
///Corr. entre lunettes et piercing = -0.0397, significative sur le train mais pas sur le valid !///


[('age', 'tatouage', 0.2974),
 ('charmes', 'tabac', 0.3001),
 ('charmes', 'piercing', 0.2568),
 ('mails', 'tatouage', 0.2706),
 ('from_europ', 'tatouage', 0.3725),
 ('paniers', 'tabac', 0.2733),
 ('lunettes', 'tatouage', 0.425)]

### Intersections 
Seules les interactions validées par plusieurs méthodes seront conservées *in fine* pour notre modèle, et ce afin de limiter le risque de surapprentissage.
<img src='vennes.png' width=600px>

In [13]:
print("Les interactions suggérées par Pearson :\n", heuristique_p.interactions_à_créer)
print("\nLes interactions suggérées par Spearman :\n", heuristique_sp.interactions_à_créer)

Les interactions suggérées par Pearson :
 [('age', 'tatouage', 0.305), ('charmes', 'tabac', 0.2607), ('mails', 'tabac', 0.2726), ('mails', 'piercing', 0.2527), ('from_europ', 'tatouage', 0.3451), ('paniers', 'tabac', 0.2527), ('lunettes', 'tatouage', 0.3944)]

Les interactions suggérées par Spearman :
 [('age', 'tatouage', 0.2974), ('charmes', 'tabac', 0.3001), ('charmes', 'piercing', 0.2568), ('mails', 'tatouage', 0.2706), ('from_europ', 'tatouage', 0.3725), ('paniers', 'tabac', 0.2733), ('lunettes', 'tatouage', 0.425)]


In [14]:
def intersection(interactions1, interactions2):
    interactions1 = {(x[0], x[1]) for x in interactions1}
    interactions2 = {(x[0], x[1]) for x in interactions2}
    return interactions1.intersection(interactions2)

In [15]:
# Les intersections communes aux deux méthodes.
intersection(heuristique_p.interactions_à_créer, 
             heuristique_sp.interactions_à_créer)

{('age', 'tatouage'),
 ('charmes', 'tabac'),
 ('from_europ', 'tatouage'),
 ('lunettes', 'tatouage'),
 ('paniers', 'tabac')}

Ces interactions sont suggérées par leur forte corrélation (Pearson **et** Spearman) avec la cible. Reste à les valider si elles ont du *sens*.

### Table de vérité
Pour justifier la pertinence des interactions créées, regardons la table de vérité de 2 variables : `lunettes`et `tatouage`. Cette table est déterminée <u>sur notre exemple</u>, car on connait parfaitement le processus générateur de la cible `label`.

In [16]:
# Table de vérité théorique : 
TdV = pd.DataFrame({'lunettes':[0, 0, 1, 1], 
                    'tatouage':[0, 1, 0, 1], 
                    'lunettes-tatouage':[0, -1, 1, 0], 
                    'label':[0, 0, 1, 0]})

print("Cor(lunettes, label) = %.2f"%np.corrcoef(TdV['lunettes'], TdV.label)[0,1])
print("Cor(tatouage, label) = %.2f"%np.corrcoef(TdV['tatouage'], TdV.label)[0,1])
print("Cor(lunette-tatouage, label) = %.2f"%np.corrcoef(TdV['lunettes-tatouage'], TdV.label)[0,1])
TdV

Cor(lunettes, label) = 0.58
Cor(tatouage, label) = -0.58
Cor(lunette-tatouage, label) = 0.82


,lunettes,tatouage,lunettes-tatouage,label
0,0,0,0,0
1,0,1,-1,0
2,1,0,1,1
3,1,1,0,0


In [17]:
# Constatation empirique : 
print("Cor(lunette, cible) = %.2f"%np.corrcoef(train['lunettes'], train.label)[0,1])
print("Cor(tatouage, cible) = %.2f"%np.corrcoef(train['tatouage'], train.label)[0,1])
print("Cor(lunette-tatouage, cible) = %.2f"%np.corrcoef(train.lunettes-train.tatouage, train.label)[0,1])

Cor(lunette, cible) = 0.13
Cor(tatouage, cible) = -0.37
Cor(lunette-tatouage, cible) = 0.38
